<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**File** { display-mode: "form" }
import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from google.colab import drive
import json
import time
import re
from IPython.display import clear_output

def parse_fraction(fraction_str):
    """Преобразует строку дроби (например, '25/1' или '25:1') в число с плавающей точкой."""
    if not fraction_str or fraction_str in ['N/A', '0/0']:
        print(f"Невалидное значение частоты кадров: '{fraction_str}'. Возвращается 0.")
        return 0
    if '/' in fraction_str:
        try:
            numerator, denominator = map(int, fraction_str.split('/'))
            return numerator / denominator if denominator != 0 else 0
        except ValueError:
            print(f"Ошибка: Неверный формат дроби '{fraction_str}'. Возвращается 0.")
            return 0
    elif ':' in fraction_str:
        try:
            numerator, denominator = map(int, fraction_str.split(':'))
            return numerator / denominator if denominator != 0 else 0
        except ValueError:
            print(f"Ошибка: Неверный формат дроби '{fraction_str}'. Возвращается 0.")
            return 0
    try:
        return float(fraction_str)
    except ValueError:
        print(f"Ошибка: Неверный формат '{fraction_str}'. Возвращается 0.")
        return 0

def get_fps_fallback(file_path):
    """Вычисляет FPS через nb_frames и duration, если avg_frame_rate некорректен."""
    try:
        cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames,duration -of json "{file_path}"'
        output = subprocess.check_output(cmd, shell=True).decode('utf-8')
        data = json.loads(output)
        stream = data['streams'][0]
        nb_frames = float(stream.get('nb_frames', 0))
        duration = float(stream.get('duration', 0))
        if nb_frames > 0 and duration > 0:
            fps = nb_frames / duration
            print(f"Вычисленный FPS (nb_frames/duration): {fps:.2f}")
            return fps
        return 0
    except Exception as e:
        print(f"Ошибка при вычислении FPS: {e}")
        return 0

def get_input_video(file_name=None):
    if file_name and os.path.exists(file_name):
        full_path = file_name
    else:
        print("No file selected or file does not exist.")
        return None

    # Получаем информацию о видео с помощью ffprobe (без sample_aspect_ratio)
    cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=width,height,avg_frame_rate,r_frame_rate -of csv=s=x:p=0 "{full_path}"'
    try:
        output = subprocess.check_output(cmd, shell=True).decode('utf-8').strip().split('x')
    except subprocess.CalledProcessError as e:
        print(f"Ошибка ffprobe: {e}")
        return None

    if len(output) >= 4:
        width, height, avg_fps_str, r_fps_str = int(output[0]), int(output[1]), output[2], output[3]

        # Пробуем avg_frame_rate
        fps = parse_fraction(avg_fps_str)
        print(f"Debug: avg_frame_rate={avg_fps_str}, calculated FPS={fps:.2f}")

        # Если FPS невалидный или нереалистичный (< 10), пробуем r_frame_rate
        if fps <= 0 or fps < 10:
            fps = parse_fraction(r_fps_str)
            print(f"Debug: r_frame_rate={r_fps_str}, calculated FPS={fps:.2f}")

        # Если FPS всё ещё невалидный или нереалистичный, вычисляем через nb_frames и duration
        if fps <= 0 or fps < 10:
            fps = get_fps_fallback(full_path)

        # Если FPS всё ещё 0, устанавливаем значение по умолчанию
        if fps == 0:
            fps = 25.0  # Значение по умолчанию, если FPS не удалось определить
            print("FPS не удалось определить, установлено значение по умолчанию: 25.0")

        # Получаем количество аудиодорожек
        audio_cmd = f'ffprobe -v error -select_streams a -show_entries stream=index -of csv=p=0 "{full_path}"'
        audio_output = subprocess.check_output(audio_cmd, shell=True).decode('utf-8').strip()
        audio_tracks = len(audio_output.split('\n')) if audio_output else 0

        # Получаем количество дорожек субтитров
        subtitle_cmd = f'ffprobe -v error -select_streams s -show_entries stream=index -of csv=p=0 "{full_path}"'
        subtitle_output = subprocess.check_output(subtitle_cmd, shell=True).decode('utf-8').strip()
        subtitle_tracks = len(subtitle_output.split('\n')) if subtitle_output else 0

        print(f"Найдено видео: {os.path.basename(full_path)}")
        print(f"Размер: {width}x{height}, FPS: {fps:.2f}")
        print(f"Аудиодорожек: {audio_tracks}")
        print(f"Дорожек субтитров: {subtitle_tracks}")

        return {
            'path': full_path,
            'filename': os.path.basename(full_path),
            'width': width,
            'height': height,
            'fps': fps,
            'audio_tracks': audio_tracks,
            'subtitle_tracks': subtitle_tracks
        }
    else:
        print("Не удалось получить информацию о видео.")
        return None

# Интерфейс для загрузки файла
upload_option = "Load from Google Drive Root"  #@param ["Upload from PC", "Load from Google Drive Root", "Load from Google Drive"]

file_name = None
input_video = None
buttons = []

def reset_button_colors():
    for btn in buttons:
        btn.style.button_color = None

output = widgets.Output()

if upload_option == "Upload from PC":
    with output:
        print("Please upload a video file.")
        uploaded = files.upload()
        if uploaded:
            file_name = list(uploaded.keys())[0]
            os.rename(file_name, os.path.join('/content', file_name))
            file_name = os.path.join('/content', file_name)
            print(f"Uploaded file: {file_name}")
            input_video = get_input_video(file_name)
        else:
            print("No file uploaded.")
            file_name = None
            input_video = None
    display(output)

elif upload_option == "Load from Google Drive Root":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.wmv']
    files_list = []

    for f in os.listdir(root_dir):
        if os.path.isfile(os.path.join(root_dir, f)) and os.path.splitext(f)[1].lower() in video_extensions:
            files_list.append(f)

    if not files_list:
        with output:
            print("No video files found in Google Drive root.")
            file_name = None
            input_video = None
        display(output)
    else:
        with output:
            print("Select a video file from Google Drive root:")

        def on_button_clicked(b):
            global file_name, input_video
            with output:
                clear_output()
                reset_button_colors()
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                    print(f"Selected file: {file_name}")
                    input_video = get_input_video(file_name)  # Запускаем обработку сразу
                else:
                    b.style.button_color = 'red'
                    print("File does not exist.")
                    input_video = None

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

elif upload_option == "Load from Google Drive":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov', '.wmv']
    files_list = []

    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in video_extensions:
                relative_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                files_list.append(relative_path)

    if not files_list:
        with output:
            print("No video files found in Google Drive or its subfolders.")
            file_name = None
            input_video = None
        display(output)
    else:
        with output:
            print("Select a video file from Google Drive (including subfolders):")

        def on_button_clicked(b):
            global file_name, input_video
            with output:
                clear_output()
                reset_button_colors()
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                    print(f"Selected file: {file_name}")
                    input_video = get_input_video(file_name)  # Запускаем обработку сразу
                else:
                    b.style.button_color = 'red'
                    print("File does not exist.")
                    input_video = None

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

In [ ]:
#@title ##**Config** { display-mode: "form" }
import os.path

height = "original" #@param ["original", "480", "720", "1080", "2160"]
video_codec = "libx264" #@param ["libx264", "libx265", "vp9", "av1"]
output_format = "mp4" #@param ["mp4", "mkv", "webm"]
video_quality = "high" #@param ["low", "medium", "high", "auto"]
encoding_mode = "hardware" #@param ["software", "hardware"]

def select_conversion_params(input_video, height=height, video_codec=video_codec, output_format=output_format, video_quality=video_quality, encoding_mode=encoding_mode):
    if not input_video:
        print("Сначала выберите входное видео.")
        return None

    # Определение целевой высоты
    if height == "original":
        target_height = input_video['height']
    else:
        target_height = int(height)

    original_ratio = input_video['width'] / input_video['height']
    target_width = int(target_height * original_ratio)
    if target_width % 2 != 0:
        target_width += 1

    # Формирование имени выходного файла
    base_name = os.path.splitext(input_video['filename'])[0]
    output_filename = f"{base_name}_{target_height}p_{video_codec}_{video_quality}_{encoding_mode}.{output_format}"
    output_path = os.path.join(os.path.dirname(input_video['path']), output_filename)

    # Формирование словаря параметров
    params = {
        'target_width': target_width,
        'target_height': target_height,
        'video_codec': video_codec,
        'output_format': output_format,
        'video_quality': video_quality,
        'encoding_mode': encoding_mode,
        'output_path': output_path
    }

    # Вывод информации о параметрах
    print(f"\nПараметры конвертации:")
    print(f"Размер: {target_width}x{target_height}")
    print(f"Кодек: {video_codec}")
    print(f"Формат: {output_format}")
    print(f"Качество: {video_quality}")
    print(f"Режим кодирования: {encoding_mode}")
    print(f"Выходной файл: {output_filename}")

    return params

# Исполнение функции
conversion_params = select_conversion_params(input_video, height, video_codec, output_format, video_quality, encoding_mode)

In [ ]:
#@title ##**Run** { display-mode: "form" }
import subprocess
import time
import re
from IPython.display import clear_output

def convert_video(input_video, params):
    if not input_video or not params:
        print("Необходимо сначала выбрать видео и параметры конвертации.")
        return

    # Проверка поддержки NVENC (для hardware режима)
    nvenc_supported = False
    if params['encoding_mode'] == 'hardware':
        try:
            # Проверяем, поддерживает ли FFmpeg h264_nvenc
            check_cmd = 'ffmpeg -encoders'
            encoders = subprocess.check_output(check_cmd, shell=True, stderr=subprocess.STDOUT).decode('utf-8')
            if 'h264_nvenc' in encoders:
                nvenc_supported = True
                print("NVENC поддерживается, будет использовано аппаратное кодирование.")
            else:
                print("NVENC не поддерживается, переключаемся на программное кодирование.")
                params['encoding_mode'] = 'software'
        except subprocess.CalledProcessError:
            print("Ошибка проверки NVENC, переключаемся на программное кодирование.")
            params['encoding_mode'] = 'software'

    # Получаем список всех потоков
    streams_cmd = f'ffprobe -v error -show_entries stream=index,codec_type -of csv=p=0 "{input_video["path"]}"'
    streams_output = subprocess.check_output(streams_cmd, shell=True).decode('utf-8').strip().split('\n')

    # Получаем аудиокодек
    audio_codec_cmd = f'ffprobe -v error -select_streams a:0 -show_entries stream=codec_name -of csv=p=0 "{input_video["path"]}"'
    audio_codec = None
    try:
        audio_codec = subprocess.check_output(audio_codec_cmd, shell=True).decode('utf-8').strip()
    except subprocess.CalledProcessError:
        pass

    # Получаем видеобитрейт (для auto)
    video_bitrate = None
    if params['video_quality'] == 'auto':
        video_bitrate_cmd = f'ffprobe -v error -select_streams v:0 -show_entries stream=bit_rate -of csv=p=0 "{input_video["path"]}"'
        try:
            video_bitrate = subprocess.check_output(video_bitrate_cmd, shell=True).decode('utf-8').strip()
            video_bitrate = int(video_bitrate) // 1000
            print(f"Видеобитрейт исходного файла: {video_bitrate} кбит/с")
        except (subprocess.CalledProcessError, ValueError):
            video_bitrate = 1000
            print(f"Не удалось определить видеобитрейт, используется: {video_bitrate} кбит/с")

    # Получаем аудиобитрейт (для auto)
    audio_bitrate = None
    if params['video_quality'] == 'auto':
        audio_bitrate_cmd = f'ffprobe -v error -select_streams a:0 -show_entries stream=bit_rate -of csv=p=0 "{input_video["path"]}"'
        try:
            audio_bitrate = subprocess.check_output(audio_bitrate_cmd, shell=True).decode('utf-8').strip()
            audio_bitrate = int(audio_bitrate) // 1000
            print(f"Аудиобитрейт исходного файла: {audio_bitrate} кбит/с")
        except (subprocess.CalledProcessError, ValueError):
            audio_bitrate = 128
            print(f"Не удалось определить аудиобитрейт, используется: {audio_bitrate} кбит/с")

    # Список совместимых аудиокодеков для MP4
    mp4_compatible_audio_codecs = ['aac', 'mp3', 'ac3']

    # Проверка поддержки субтитров
    format_supports_subtitles = True
    if params["output_format"] == "mp4" and input_video.get('subtitle_tracks', 0) > 0:
        print("Внимание: MP4 имеет ограниченную поддержку субтитров. Рекомендуется MKV.")
        format_supports_subtitles = False

    # Определяем битрейты для low, medium, high
    if params['video_quality'] != 'auto':
        quality_settings = {
            'low': {'target_bitrate': 5000, 'max_bitrate': 7500},
            'medium': {'target_bitrate': 8000, 'max_bitrate': 12000},
            'high': {'target_bitrate': 16000, 'max_bitrate': 24000}
        }
        resolution_factor = (params['target_width'] * params['target_height']) / (1920 * 1080)
        target_bitrate = int(quality_settings[params['video_quality']]['target_bitrate'] * resolution_factor)
        max_bitrate = int(quality_settings[params['video_quality']]['max_bitrate'] * resolution_factor)
        target_bitrate = max(target_bitrate, 500)
        max_bitrate = max(max_bitrate, 750)
        print(f"Качество {params['video_quality']}: Целевой битрейт {target_bitrate} кбит/с, Макс. битрейт {max_bitrate} кбит/с")
    else:
        target_bitrate = video_bitrate
        max_bitrate = None

    # Формируем команду ffmpeg
    cmd = f'ffmpeg -y -i "{input_video["path"]}" '

    # Видео параметры
    cmd += f'-map 0 '
    if params['encoding_mode'] == 'hardware':
        if params['video_codec'] == 'libx264':
            cmd += f'-c:v h264_nvenc '
        elif params['video_codec'] == 'libx265':
            cmd += f'-c:v hevc_nvenc '
        else:
            print(f"Аппаратное кодирование не поддерживается для {params['video_codec']}, переключаемся на программное.")
            cmd += f'-c:v {params["video_codec"]} '
            params['encoding_mode'] = 'software'
    else:
        cmd += f'-c:v {params["video_codec"]} '

    if params['video_quality'] == 'auto':
        cmd += f'-b:v {target_bitrate}k '
        if params['encoding_mode'] == 'software':
            cmd += f'-preset slow '
    else:
        cmd += f'-b:v {target_bitrate}k -maxrate {max_bitrate}k -bufsize {max_bitrate * 2}k '
        if params['encoding_mode'] == 'software':
            cmd += f'-preset slow -profile:v high -level 4.1 '
        else:
            cmd += f'-preset p7 '  # Максимальное качество для NVENC
    cmd += f'-vf "scale={params["target_width"]}:{params["target_height"]}:force_original_aspect_ratio=decrease" '
    cmd += f'-r {input_video["fps"]} '

    # Аудио
    if input_video.get('audio_tracks', 0) > 0 and audio_codec:
        if params["output_format"] == "mp4" and audio_codec not in mp4_compatible_audio_codecs:
            if params['video_quality'] == 'auto':
                cmd += f'-c:a aac -b:a {audio_bitrate}k -ar 48000 -ac 2 '
                print(f"Аудиокодек '{audio_codec}' несовместим с MP4. Перекодируется в AAC с битрейтом {audio_bitrate} кбит/с.")
            else:
                cmd += f'-c:a aac -b:a 320k -ar 48000 -ac 2 '
                print(f"Аудиокодек '{audio_codec}' несовместим с MP4. Перекодируется в AAC с битрейтом 320 кбит/с.")
        else:
            cmd += '-c:a copy '
            print(f"Аудиокодек '{audio_codec}' совместим, копируется.")
    else:
        cmd += '-an '
        print("Аудиопотоков нет, аудио отключается.")

    # Субтитры
    if input_video.get('subtitle_tracks', 0) > 0:
        if format_supports_subtitles:
            cmd += '-c:s copy '
        else:
            cmd += '-c:s mov_text '
    else:
        cmd += '-sn '

    # Оптимизация для стриминга
    cmd += f'-movflags +faststart "{params["output_path"]}"'

    print("Запуск конвертации...")
    print(f"Команда: {cmd}")

    # Запуск конвертации
    start_time = time.time()
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)

    # Прогресс
    print("Прогресс конвертации:")
    duration_pattern = re.compile(r"Duration: (\d{2}):(\d{2}):(\d{2})\.\d+")
    time_pattern = re.compile(r"time=(\d{2}):(\d{2}):(\d{2})\.\d+")
    duration_seconds = None

    for line in process.stdout:
        if not duration_seconds:
            match = duration_pattern.search(line)
            if match:
                h, m, s = map(int, match.groups())
                duration_seconds = h * 3600 + m * 60 + s

        match = time_pattern.search(line)
        if match and duration_seconds:
            h, m, s = map(int, match.groups())
            current_seconds = h * 3600 + m * 60 + s
            progress = min(100, int(current_seconds / duration_seconds * 100))
            elapsed = time.time() - start_time
            clear_output(wait=True)
            print(f"Прогресс: {progress}% | Прошло времени: {int(elapsed)} сек")
            print(line.strip())
        else:
            print(line.strip())

    # Завершение
    process.wait()
    total_time = time.time() - start_time

    if process.returncode == 0:
        print(f"\nКонвертация завершена за {int(total_time)} секунд.")

        # Проверка потоков
        check_cmd = f'ffprobe -v error -show_entries stream=index,codec_type -of csv=p=0 "{params["output_path"]}"'
        check_output = subprocess.check_output(check_cmd, shell=True).decode('utf-8').strip().split('\n')

        print(f"В исходном файле: {len(streams_output)} потоков")
        print(f"В конвертированном файле: {len(check_output)} потоков")

        video_streams = sum(1 for line in check_output if 'video' in line)
        audio_streams = sum(1 for line in check_output if 'audio' in line)
        subtitle_streams = sum(1 for line in check_output if 'subtitle' in line)

        print(f"Видео потоков: {video_streams}")
        print(f"Аудио потоков: {audio_streams}")
        print(f"Субтитры: {subtitle_streams}")

        # Битрейт выходного файла
        bitrate_cmd = f'ffprobe -v error -show_entries format=bit_rate -of csv=p=0 "{params["output_path"]}"'
        try:
            bitrate_output = subprocess.check_output(bitrate_cmd, shell=True).decode('utf-8').strip()
            bitrate_kbps = int(bitrate_output) // 1000
            print(f"Общий битрейт выходного файла: {bitrate_kbps} кбит/с")
        except Exception as e:
            print(f"Не удалось определить битрейт: {e}")

        print(f"Файл сохранен: {params['output_path']}")
        return params['output_path']
    else:
        print("\nОшибка конвертации.")
        return None

# Исполнение
output_video = convert_video(input_video, conversion_params)

In [ ]:
#@title ##**Download** { display-mode: "form" }

def download_video(video_path):
  if not video_path or not os.path.exists(video_path):
    print("Файл для скачивания не найден.")
    return

  try:
    from google.colab import files
    print("Скачивание файла. Это может занять некоторое время в зависимости от размера видео...")
    files.download(video_path)
    print("Скачивание начато.")
  except Exception as e:
    print(f"Ошибка при скачивании: {e}")
    print("Вы можете скачать файл вручную из вашего Google Drive.")

# Исполнение функции
if output_video:
  download_video(output_video)